# Cuaderno 32: Parámetros asociados a heurísticas

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

En este cuaderno revisitaremos la formulación basada en desigualdades de corte para el problema del agente viajero asimétrico y exploraremos cómo los parámetros de configuración del solver Gurobi relacionados con la aplicación de heurísticas primales pueden ser usados para mejorar el desempeño del método de branch-and-cut.

Dados: 
* un grafo dirigido **completo** $D=(V,A)$, con $V=\{1, \ldots, n\}$; y,
* un vector $c \in \ZZ^{A}$ de costos asociados a los arcos de $D$.

El *problema del agente viajero asimétrico (Asymmetric Traveling Salesman Problem, ATSP)* consiste en encontrar un circuito dirigido que visite **todos** los nodos de $D$ y que tenga el menor costo posible.

En el Cuaderno 15 presentamos la siguiente formulación para el ATSP, basada en desigualdades de corte:

\begin{align*} 
\min &\sum_{(i,j) \in A} c_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
&\sum_{(j, i) \in A} x_{ji} = 1, \quad \forall i \in V,\\
&\sum_{(i, j) \in A} x_{ij} = 1, \quad \forall i \in V,\\ 
&\sum_{(i,j) \in \delta^{-}(W)} x_{ij} \geq 1, \quad \forall W \subset V, \emptyset \neq W \neq V,\\
& x_{ij} \in \{0, 1\}, \quad \forall (i, j) \in A.
\end{align*}

La función objetivo mide el costo total de los arcos seleccionados en el tour.

Las dos primeras familias de restricciones corresponden a las restricciones de grado de los nodos.

La tercera familia incluye las restricciones de corte para eliminación de subtoures. Esta familia contiene un número exponencial de desigualdades, que deben implementarse como restricciones tipo lazy.

Se conoce que esta formulación del problema está asociada a cotas duales ajustadas. Por otra parte, para instancias grandes, se torna difícil encontrar buenas soluciones factibles, por lo que resulta natural ajustar los parámetros del solver para intensificar la aplicación de heurísticas primales.


Definimos primero los datos. Usaremos la función `randint` para generar puntos con coordenadas aleatorias en el rango {0,..,100}. El costo de un arco $(i,j)$ será igual a la distancia euclideana entre $i$ y $j$:

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import random as rm
import math 

# Número de nodos del grafo
n = 200

# Nodos del grafo
V = gp.tuplelist(range(1,n+1))

# Nodos sin el primero
V2 = V[1:]

# Posiciones de los nodos en un plano euclideano entre (0,0) y (100, 100)
coordx={i : rm.randint(0,100) for i in V}
coordy={i : rm.randint(0,100) for i in V}

# los costos son las distancias eculideanas
c = gp.tupledict({
    (i,j) : math.sqrt((coordx[i] - coordx[j])**2 + (coordy[i] - coordy[j])**2)
    for i in V for j in V if i!=j
})
 
A = c.keys()


Emplearemos el módulo `matplotlib` para graficar el tour de la solución. Definiremos para ello la función `dibujarTour` que recibe tres argumentos: una lista `coordx` con las coordenadas horizontales de los nodos, una lista `coordy` con las coordenadas verticales y un vector `tour` con una permutación de los nodos indicando el orden de visita en la solución.

In [ ]:
import matplotlib.pyplot as plt
import random

def dibujarTour(coordx, coordy, tour):
    Tx = [coordx[i] for i in tour]
    Ty = [coordy[i] for i in tour]
    plt.plot(Tx[:-1], Ty[:-1], 'ro')
    for i in range(len(tour)-1):
        s='{}'.format(tour[i])
        plt.text(Tx[i],Ty[i]+1,s)
        plt.arrow(Tx[i], Ty[i], Tx[i+1]-Tx[i], Ty[i+1]-Ty[i], color='blue', 
                  length_includes_head=True, width=0.1, head_width=2)
    display(plt.show())


Definimos ahora el objeto modelo, las variables, la función objetivo y las restricciones de grado. Adicionalmente, creamos propiedades en el objeto modelo para tener acceso a la lista de nodos, lista de arcos y diccionario de variables desde la función callback que implementaremos para la separación de las restricciones de corte.

In [ ]:
# Crear el objeto modelo
m = gp.Model('atsp-corte-lazy')

# Crear las variables
x = m.addVars(A, name="x", vtype=GRB.BINARY)

# Crear la función objetivo
m.setObjective(x.prod(c,'*'), GRB.MINIMIZE)

# Restricciones de grado saliente
m.addConstrs((x.sum(i,'*') == 1 for i in V), "g_saliente");

# Restricciones de grado entrante
m.addConstrs((x.sum('*', i) == 1 for i in V), "g_entrante");

# Crear variables en el objeto modelo para tener acceso a x, V y A
m._x = x
m._V = V
m._A = A

Definimos la función `corte_sal_minimo` que calcula el corte saliente de capacidad mínima en un grafo dirigido. Esta función será utilizada por el callback para resolver el problema de separación con soluciones fraccionarias.

In [ ]:
import networkx as nx

# corte saliente de capacidad minima
def corte_sal_minimo(V, u):
    # Parametros:
    # V: lista con los nodos del grafo
    # u: diccionario indexado por los arcos del grafo, con sus capacidades
    # se fija un nodo s arbitrario
    s = V[0]
    V2 = [i for i in V if i!=s]
    G = nx.DiGraph()
    G.add_weighted_edges_from([(i, j, u[i,j]) for (i,j) in u.keys()], weight='u')
    # inicializar dmin con la suma de capacidades de todos los arcos
    dmin = sum([u[i,j] for (i, j) in u.keys()])
    hay_solucion = False
    # se calculan los cortes minimos (s, t) y (t, s) para todo t != s
    for t in V2: 
        d1, (W1, W1c) = nx.minimum_cut(G, s, t, capacity='u')
        d2, (W2, W2c) = nx.minimum_cut(G, t, s, capacity='u')
        if d1 < d2 and d1 < dmin:
            dmin, Wmin, Wminc = d1, W1, W1c
            hay_solucion = True
        elif d2 <= d1 and d2 < dmin:
            dmin, Wmin, Wminc = d2, W2, W2c
            hay_solucion = True
    if  not hay_solucion:
        print('*** Error:')
        print(V)
        for (i,j) in [(i, j) for (i, j) in u.keys() if i==1]:
            print([(1,j), u[1,j]])
    return dmin, Wmin, Wminc


Finalmente, implementamos la función callback para separar las restricciones de corte a partir de soluciones enteras (`where==MIPSOL`) o de soluciones fraccionarias (`where==MIPNODE`).

In [ ]:
# Función callback para separar desigualdades de corte
def mycallback(model, where):
    # Esta función se activará cuando se encuentre una 
    # nueva solución entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solucion actual
        vx = model.cbGetSolution(model._x)
        # Determinar los arcos seleccionados en la solucion
        L = gp.tuplelist([(i,j) for (i,j) in model._A if vx[i,j]>0.1])
        # Construir la lista W de nodos que pueden ser alcanzados desde 1
        W = [1]
        i = 1
        while True :
            # seleccionar el único arco saliente de i en L
            a = L.select(i,'*')[0]
            L.remove(a)
            if a[1]==1: 
                break
            W.append(a[1])
            i = a[1]
        # Si W!=V, agregar la desigualdad de corte asociada a W
        if len(W)!=len(model._V):
            Wc = [i for i in model._V if i not in W]
            model.cbLazy(model._x.sum(W, Wc) >= 1)
    # Esta funcion se activará cuando se encuentre 
    # la nueva solución optima en un nodo
    elif where == GRB.Callback.MIPNODE:
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            # Recuperar los valores de la solucion (fraccionaria) actual
            vx = model.cbGetNodeRel(model._x)
            # encontrar el corte saliente de capacidad minima
            u, W, Wc = corte_sal_minimo(model._V, vx)
            # Si la capacidad de este corte es inferior a 1, agregar nueva desigualdad lazy
            if u <= 0.99:
                model.cbLazy(model._x.sum(W, Wc) >= 1)


Establecemos un tiempo límite para la solución del modelo y una tolerancia para la brecha de optimalidad. Adicionalmente, indicamos a Gurobi que estamos utilizando un modelo con restricciones lazy.

In [ ]:
# Terminar al alcanzar un Gap del 1%
m.Params.MIPGap = 0.01

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180

# Configurar Gurobi para usar restricciones lazy
m.Params.LazyConstraints = 1

Construimos una solución inicial con la heurística de mejor inserción examinada en el Cuaderno 29:

In [ ]:
# Heurística de mejor insercion (best-insertion)
# construir una lista T con el orden de visita de los nodos en el tour
# Tour inicial:
T = [1, 2, 1]
# Nodos por procesar:
W = [i for i in V if not i in [1,2]]
# Constante suficientemente alta
# Repetir mientras W contenga nodos
while W!=[]:
    # construir una lista L con tuplas (c_i, j_i, i) donde:
    # c_i : mejor costo de insercion de i en el tour T
    # j_i : mejor posicion de insercion de i en el tour T, 1 <= j_i <= |T|-1
    L = []
    for i in W:
        # construimos primero una lista L_i con tuplas (c_ik, k)
        # c_ik: costo de insertar i en la posicion k
        Li = [(c[T[k-1],i] + c[i, T[k]] - c[T[k-1], T[k]], k) for k in range(1, len(T)-1)]
        # agregamos el elemento de Li de costo minimo a L
        # recordar que para ordenar tuplas se emplear por defecto el criterio lexicográfico
        ci, ji = min(Li)
        L.append((ci, ji, i))
    # determinar el elemento de insercion mas barata, y su posicion
    (delta, j, i) = min(L)
    # insertar i en T en la posicion j
    T.insert(j, i)
    # eliminar i de W
    W.remove(i)

print(T)
print(sum([c[T[i],T[i+1]] for i in range(0, len(T)-1)]))
       

Mejoramos la solución anterior con una heurística 3-OPT descrita en el Cuaderno 29:

In [ ]:
# Mejorar T usando 3-OPT
# este lazo se repite mientras se encuentren mejoras
hay_mejoras = True
while hay_mejoras:
    hay_mejoras = False
    print('*')
    for k1 in range(1, len(T)-5):
        for k2 in range(k1+2, len(T)-3):
            for k3 in range(k2+2, len(T)-1):
                # determinar variacion del costo al ejecutar el intercambio:
                delta = c[T[k1], T[k2+1]] + c[T[k2], T[k3+1]] + c[T[k3], T[k1+1]] \
                      - c[T[k1], T[k1+1]] - c[T[k2], T[k2+1]] - c[T[k3], T[k3+1]]
                # si la variacion es negativa, realizar el intercambio
                if delta < -0.01:
                    T = T[0:k1+1] + T[k2+1:k3+1] + T[k1+1:k2+1] + T[k3+1:len(T)]
                    hay_mejoras = True
            
print (T)
print(sum([c[T[i],T[i+1]] for i in range(0, len(T)-1)]))


Agregamos la solución inicial al modelo:

In [ ]:
# originalmente, fijamos el valor de inicio de todas las variables a cero:
for i,j in A:
    x[i,j].setAttr('Start', 0.0)

# luego  fijar a 1 las variables asociadas a los arcos del tour T
for k in range(1, len(T)):
    x[T[k-1], T[k]].setAttr('Start', 1.0)

# es necesario llamar a update() luego de cambiar los atributos de variables, de restricciones o del modelo
m.update()

Finalmente, resolvemos el modelo.

In [ ]:
# Resolver el modelo
m.optimize(mycallback)


## Parámetros de Gurobi

Como se indicó en el Cuaderno 28, el comportamiento de los algoritmos de solución implementados en Gurobi puede ser controlado a través de distintos *parámetros*. Examinaremos a continuación algunos parámetros que controlan la aplicación de las heurísticas primales dentro del método de branch-and-cut.


### MIPFocus

Recordemos que el parámetro [`MIPFocus`](https://www.gurobi.com/documentation/10.0/refman/mipfocus.html) controla estrategia general de solución del método de branch-and-bound, entre otras cosas a través del balance entre el tiempo dedicado a la generación de cortes y a las heurísticas primales. Por defecto, este parámetro tiene el valor de 0, y puede recibir valores enteros entre 0 y 3.

Asignando un valor de `MIPFocus = 1` se intensifica la búsqueda de soluciones factibles.

In [ ]:
# Eliminar información de solución anterior
m.reset()

# Enfocar estrategia en mejoramiento de la cota dual
m.Params.MIPFocus = 1

# Resolver el modelo
m.optimize(mycallback)

### Heuristics

El parámetro [`Heuristics`](https://www.gurobi.com/documentation/10.0/refman/heuristics.html) indica el porcentaje del tiempo de cálculo que Gurobi debería emplear en la ejecución de heurísticas primales. Este parámetro puede tomar valores decimales entre 0 y 1. El valor por defecto es 0.05 (5% del tiempo de cálculo dedicado a heurísticas).

In [ ]:
# Descartar información de solución anterior
m.reset()

# Fijar el uso de heurísticas en el 50%
m.Params.Heuristics = 0.5

# Resolver el modelo
m.optimize(mycallback)

### NoRelHeurTime

La heurística `NoRel` intenta encontrar soluciones factibles de alta calidad *antes* de empezar a resolver la relajación lineal del modelo. Esta heurística se puede utilizar para proporcionar una buena solución inicial al modelo.

El parámetro [`NoRelHeurTime`](https://www.gurobi.com/documentation/10.0/refman/norelheurtime.html) controla el tiempo (en segundos) que Gurobi puede dedicar a la ejecución de la heurística `NoRel`. Este parámetro puede tomar valores decimales no negativos. El valor por defecto es 0 (heurística `NoRel` desactivada).

In [ ]:
# Restaurar el parámetro Heuristics al valor por defecto
m.Params.Heuristics = 0.05

# Enfocar estrategia en mejorar la cota dual
m.Params.NoRelHeurTime = 150

# Descartar información de solución anterior
m.reset()

# Resolver el modelo
m.optimize(mycallback)

### NoRelHeurWork

El uso de la heurística `NoRel` a través del parámetro `NoRelHeurTime` introduce aleatoriedad en la solución del modelo. Si se desea mantener un comportamiento determinístico, es mejor usar al parámetro [`NoRelHeurWork`](https://www.gurobi.com/documentation/10.0/refman/norelheurwork.html) en su lugar. Este parámetro indica la cantidad de trabajo dedicada a la heurística `NoRel` (cada unidad de trabajo corresponde aproximadamente a un segundo de cálculo, pero esto puede variar de acuerdo a la máquina, el número de núcleos y en algunos casos incluso el modelo). El parámetro puede tomar valores decimales no negativos. El valor por defecto es 0 (heurística `NoRel` desactivada).

In [ ]:
# Regresar NoRelHeurTime a su valor por defecto
m.Params.NoRelHeurTime = 0

# Fijar 150 unidades de trabajo para heurística NoRel
m.Params.NoRelHeurWork = 20

# Descartar información de solución anterior
m.reset()

# Resolver el modelo
m.optimize(mycallback)

### ImproveStartTime, ImproveStartNodes e ImproveStartGap

Es posible indicar a Gurobi que, a partir de un cierto momento, cambie la estrategia de búsqueda para abandonar el mejoramiento de la cota y concentrar todos los esfuerzos en la generación de soluciones factibles de mejor calidad.

Los parámetros `ImproveStartTime`, `ImproveStartNodes` e `ImproveStartGap` permiten especificar tres criterios distintos para indicar cuándo debe ocurrir este cambio de estrategia.

El parámetro [`ImproveStartTime`](https://www.gurobi.com/documentation/10.0/refman/improvestarttime.html) se emplea para indicar que el cambio de estrategia deberá ocurrir una vez transcurrida la cantidad de segundos especificada. Este parámetro puede tomar valores decimales no negativos. El valor por defecto es infinito.

El parámetro [`ImproveStartNodes`](https://www.gurobi.com/documentation/10.0/refman/improvestartnodes.html) se emplea para indicar que el cambio de estrategia deberá ocurrir una vez que se hayan explorado la cantidad especificada de nodos del árbol de branch-and-cut. Este parámetro puede tomar valores decimales no negativos. El valor por defecto es infinito.

El parámetro [`ImproveStartGap`](https://www.gurobi.com/documentation/10.0/refman/improvestartgap.html) se emplea para indicar que el cambio de estrategia deberá ocurrir una vez que se alcance la brecha de optimalidad especificada. Este parámetro puede tomar valores decimales no negativos. El valor por defecto es cero.


In [ ]:
# Regresar NoRelHeurWork a su valor por defecto
m.Params.NoRelHeurWork = 0

# Especificar cambio a modo de búsqueda a los 30s
m.Params.ImproveStartTime = 30

# Descartar información de solución anterior
m.reset()

# Resolver el modelo
m.optimize(mycallback)

Es posible combinar varios parámetros para intentar estrategias mixtas. Por ejemplo, podemos correr la heurística `NoRel` durante 100 segundos, luego optimizar en la forma usual durante 50 segundos y finalmente cambiar al modo de mejoramiento de soluciones en el tiempo restante:

In [ ]:
# Correr heurística NoRel por 100 segundos
m.Params.NoRelHeurTime = 100 

# Especificar cambio a modo de búsqueda a los 150s
m.Params.ImproveStartTime = 50

# Descartar información de solución anterior
m.reset()

# Resolver el modelo
m.optimize(mycallback)

Finalmente, podemos graficar la solución encontrada:

In [ ]:
# Crear lista con arcos seleccionados en la solucion
vx = m.getAttr('x', x)
L = gp.tuplelist([(i,j) for i,j in A if vx[i,j]>0.1])

# Recuperar el tour como un ordenamiento de los nodos
T = [1]
# nodo actual:
i = 1
while True:
    # Determinar sucesor de i
    a = L.select(i,'*')[0]
    L.remove(a)
    # Colocar sucesor en la lista del tour y actualizar i
    T.append(a[1])
    i = a[1]
    # Terminar cuando el nodo colocado sea 1
    if i==1: 
        break;
        
# Graficar el tour
dibujarTour(coordx, coordy, T)    